<a href="https://colab.research.google.com/github/pcmbs/preset-embedding_audio-model-colab/blob/main/notebooks/TAL_NoiseMaker_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Download TAL-NoiseMaker and presets

In [1]:
!wget -c https://tal-software.com/downloads/plugins/TAL-NoiseMaker_64_linux.zip && unzip -o TAL-NoiseMaker_64_linux.zip && rm TAL-NoiseMaker_64_linux.zip

--2023-07-14 18:16:11--  https://tal-software.com/downloads/plugins/TAL-NoiseMaker_64_linux.zip
Resolving tal-software.com (tal-software.com)... 104.21.13.116, 172.67.167.238, 2606:4700:3037::6815:d74, ...
Connecting to tal-software.com (tal-software.com)|104.21.13.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7048068 (6.7M) [application/x-zip-compressed]
Saving to: ‘TAL-NoiseMaker_64_linux.zip’

TAL-NoiseMaker_64_l 100%[===================>]   6.72M  7.09MB/s    in 0.9s    

2023-07-14 18:16:13 (7.09 MB/s) - ‘TAL-NoiseMaker_64_linux.zip’ saved [7048068/7048068]

Archive:  TAL-NoiseMaker_64_linux.zip
  inflating: TAL-NoiseMaker/ReadmeLinux.txt  
  inflating: TAL-NoiseMaker/TAL-NoiseMaker.clap  
   creating: TAL-NoiseMaker/TAL-NoiseMaker.vst3/
   creating: TAL-NoiseMaker/TAL-NoiseMaker.vst3/Contents/
   creating: TAL-NoiseMaker/TAL-NoiseMaker.vst3/Contents/x86_64-linux/
  inflating: TAL-NoiseMaker/TAL-NoiseMaker.vst3/Contents/x86_64-linux/TAL-NoiseMaker.s

Install dawdreamer

In [2]:
!pip install dawdreamer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 MB 15.8 MB/s eta 0:00:00


Download and import presets

In [3]:
!gdown "1UVZaVulQH8y0_JqwN-tXsqSD14QSKSVR"

import json

with open("noisemaker_presets.json", "rb") as f:
    presets = json.load(f)

Downloading...
From: https://drive.google.com/uc?id=1UVZaVulQH8y0_JqwN-tXsqSD14QSKSVR
To: /content/noisemaker_presets.json
100% 1.31M/1.31M [00:00<00:00, 145MB/s]


download and import TAL-NoiseMaker settings for evaluation

In [4]:
!gdown "1z7LgFquYwmB1FbY0cW4u6fesRl5a150W"

import sound_properties_eval as spe

Downloading...
From: https://drive.google.com/uc?id=1z7LgFquYwmB1FbY0cW4u6fesRl5a150W
To: /content/sound_properties_eval.py
100% 2.35k/2.35k [00:00<00:00, 11.4MB/s]


# Synth info

In [5]:
import dawdreamer as daw

engine = daw.RenderEngine(44_100, 128)
# Make a processor and give it the unique name "my_synth", which we use later.
synth = engine.make_plugin_processor("TAL-NoiseMaker", "TAL-NoiseMaker/TAL-NoiseMaker.vst3")
assert synth.get_name() == "TAL-NoiseMaker"

print(f"synth name: {synth.get_name()}")
print(f"synth num inputs: {synth.get_num_input_channels()}")
print(f"synth num outputs: {synth.get_num_output_channels()}")

params_list = synth.get_parameters_description()
for param_dict in params_list[:89]: # params outside this range are not relevant
    print(f"index: {param_dict['index']}, name: {param_dict['name']}")

synth name: TAL-NoiseMaker
synth num inputs: 2
synth num outputs: 2
index: 0, name: -
index: 1, name: Master Volume
index: 2, name: Filter Type
index: 3, name: Filter Cutoff
index: 4, name: Filter Resonance
index: 5, name: Filter Keyfollow
index: 6, name: Filter Contour
index: 7, name: Filter Attack
index: 8, name: Filter Decay
index: 9, name: Filter Sustain
index: 10, name: Filter Release
index: 11, name: Amp Attack
index: 12, name: Amp Decay
index: 13, name: Amp Sustain
index: 14, name: Amp Release
index: 15, name: Osc 1 Volume
index: 16, name: Osc 2 Volume
index: 17, name: Osc 3 Volume
index: 18, name: Osc Mastertune
index: 19, name: Osc 1 Tune
index: 20, name: Osc 2 Tune
index: 21, name: Osc 1 Fine Tune
index: 22, name: Osc 2 Fine Tune
index: 23, name: Osc 1 Waveform
index: 24, name: Osc 2 Waveform
index: 25, name: Osc Sync
index: 26, name: Lfo 1 Waveform
index: 27, name: Lfo 2 Waveform
index: 28, name: Lfo 1 Rate
index: 29, name: Lfo 2 Rate
index: 30, name: Lfo 1 Amount
index: 31,

# Classes & Functions definition

class PresetRenderer: Class to rendered audio from TAL-Noisemaker using DawDreamer.


In [5]:
from pathlib import Path
from typing import List, Union, Sequence
import dawdreamer as daw
import numpy as np

class PresetRenderer:
    """
    Class to rendered audio from TAL-Noisemaker using DawDreamer.
    """

    def __init__(
        self,
        synth_path: str = "TAL-NoiseMaker/TAL-NoiseMaker.vst3",
        sample_rate: int = 44_100,
        render_duration_s: float = 4,
        fadeout_duration_s: float = 0.2,
        export_to_mono: bool = True,
        normalize_audio: bool = False,
    ):
        ### Paths and name related member variables
        self.synth_path = synth_path
        self.synth_name = Path(self.synth_path).stem

        ### DawDreamer related member variables
        self.sample_rate = sample_rate
        self.render_duration_s = render_duration_s  # rendering time in seconds
        self.engine = daw.RenderEngine(self.sample_rate, block_size = 128)  # pylint: disable=E1101
        self.synth = self.engine.make_plugin_processor(self.synth_name, self.synth_path)

        ### MIDI related member variables
        self.midi_note: int
        self.midi_note_velocity: int
        self.midi_note_start: float
        self.midi_note_duration_s: float

        ### Rendering relative member variables
        # fadeout
        self.fadeout_duration_s = fadeout_duration_s
        self.fadeout_len = int(self.sample_rate * self.fadeout_duration_s)
        # avoid multiplication with an empty array (from linspace) if no fadeout_duration_s = 0
        if self.fadeout_len > 0:
            self.fadeout = np.linspace(1, 0, self.fadeout_len)
        else:  # hard-coding if fadeout_duration_s = 0
            self.fadeout = 1.0

        # export to mono
        self.export_to_mono = export_to_mono

        # normalize
        self.normalize_audio = normalize_audio


    def assign_preset(self, preset: list[dict]) -> None:
        """
        Assign a preset to the synthesizer.
        """
        self.current_preset = preset  # update instance's current parameters

        # individually set each parameters since DawDreamer does not accept
        # list of tuples (param_idx, value)
        for param in self.current_preset:
            self.synth.set_parameter(param["index"], param["value"])

    def set_midi_parameters(
        self,
        midi_note: int = 60,
        midi_note_velocity: int = 100,
        midi_note_start: float = 0.0,
        midi_note_duration: float = 2,
    ):
        """
        Set the instance's midi parameters.
        """
        self.midi_note = midi_note
        self.midi_note_velocity = midi_note_velocity
        self.midi_note_start = midi_note_start
        self.midi_note_duration_s = midi_note_duration

        # Generate a MIDI note, specifying a start time and duration, both in seconds
        self.synth.add_midi_note(
            self.midi_note, self.midi_note_velocity, self.midi_note_start, self.midi_note_duration_s
        )

    def render_note(self) -> Sequence:
        """
        Renders a midi note (for the currently set patch) and returns the generated audio as ndarray.
        """

        if self.current_preset is None:
            raise ValueError("No preset has been set yet. Please use `assign_preset()` first.")

        graph = [(self.synth, [])]  # Generate DAG of processes
        self.engine.load_graph(graph)  # load a DAG of processors
        self.engine.render(self.render_duration_s)  # Render audio.
        audio = self.engine.get_audio()  # get audio
        if self.export_to_mono:  # convert to mono if required
            audio = np.mean(audio, axis=0, keepdims=True)
        if self.normalize_audio:  # normalize audio if required
            audio = audio / np.max(np.abs(audio))
        audio[..., -self.fadeout_len :] = audio[..., -self.fadeout_len :] * self.fadeout  # fadeout

        return audio

In [6]:
def generate_eval_audio(renderer: PresetRenderer, settings: spe.SoundProperties, num_samples: int = 10):
    renderer.assign_preset(presets[settings.preset])

    if settings.extra_params:
        for param in settings.extra_params:
            renderer.synth.set_parameter(*param)

    param_vals_for_eval = np.linspace(*settings.interval, num_samples)

    output = []
    for val in param_vals_for_eval:
        renderer.synth.set_parameter(settings.param_idx, val)
        output.append(renderer.render_note())

    return output

# Tests

In [7]:
import IPython.display as ipd

renderer = PresetRenderer()
renderer.set_midi_parameters()

current_setting = spe.reverb

output = generate_eval_audio(renderer, current_setting, 5)

for i, audio in enumerate(output):
    #ipd.display(ipd.Markdown(f"{current_setting.base_param_name}: {i}"))
    ipd.display(ipd.Audio(audio, rate=int(renderer.sample_rate), normalize=False))